Load dependencies

In [1]:
import numpy as np
import pandas as pd
import os
import sys
import json
import tensorflow as tf
from NN_Predictor import NN
import NN_Trainer
import cobra

2023-07-12 11:57:41.025208: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
from pathlib import Path
path = Path.cwd()
sys.path.append(path)
file_path = os.path.join(path.parent,'files', 'NN')
output_path = os.path.join(path.parent,'files', 'custom scores')

#Tensorflow complains because the NN is not optimized perfectly
tf.get_logger().setLevel('ERROR')

## Introduction

The purpose of this notebook is to show some examples of using the NN

## Making predictions using the NN

Load in your favourite Neural Network

In [3]:
NN_MS = NN(path=file_path+'/NN_MS_lite.npz')

Loading network at user provided path


In [4]:
file_path+'/NN_MS_lite.npz'

'/home/meine/Documents/git/DNNGIOR/files/NN/NN_MS_lite.npz'

You can also use modeltype

In [5]:
NN_MS = NN(modeltype='ModelSEED')

/home/meine/Documents/git/DNNGIOR/files/NN/NN_MS_lite.npz
Loading default ModelSEED NN


Load in the data, the NN requires a binary list indicating presences of reactions, it is important that this is in the right order. However, you can give it a dictionary and it will put it in the right order for you.

load in the dictionary:

In [7]:
with open(file_path+'/binary_example.json', 'r') as j:
    binary_example = json.loads(j.read())

and make the prediction

In [8]:
prediction = NN_MS.predict(input=binary_example)

#reactions not found in keys:  0 / 922
[[0 1 0 ... 1 0 0]]


# Multiple predictions

We can also make predictions for multiple models at the same time, starting with a reaction presence dataframe where the rows are the different reactions and the columns are the model ids:

In [13]:
#multiple binary reaction lists (csv)
input_path = file_path+'/Sample_reaction_presence.csv'
df = pd.read_csv(input_path, index_col=0)

We can then create a array of prediction scores with the same order of reaction and model ids

In [14]:
NN_MS.predict(input=df)

[[0 1 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]
 ...
 [0 1 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]
 [0 1 0 ... 1 0 0]]


,1827300.3,693748.4,1637747.3,1165841.3,912630.3,1950946.3,1378073.3,589873.29,1919105.4,1951070.3
rxn08713_c0,1.062802e-07,3.231988e-07,0.000009,1.239211e-06,6.358366e-08,1.354992e-07,1.181051e-02,4.064459e-06,1.786075e-04,1.044991e-07
rxn03137_c0,1.000000e+00,9.999979e-01,0.999837,9.930349e-01,9.978111e-01,9.546268e-01,9.999908e-01,9.982888e-01,9.681360e-01,9.639262e-01
rxn09195_c0,1.285675e-10,7.526295e-10,0.000011,1.487453e-09,2.450623e-08,3.293829e-10,2.042630e-02,1.123587e-08,8.171497e-06,2.150302e-10
rxn01279_c0,9.957535e-01,1.208297e-01,0.230936,1.282320e-03,5.543450e-04,8.808032e-07,9.481068e-02,2.959512e-04,1.846205e-03,1.596615e-06
rxn00720_c0,1.864025e-10,3.296763e-07,0.000058,1.038487e-05,6.028677e-10,5.315302e-06,3.545153e-10,9.507275e-06,5.255141e-05,4.817544e-06
...,...,...,...,...,...,...,...,...,...,...
rxn05324_c0,6.183886e-07,1.000000e+00,0.999984,9.999784e-01,1.683694e-06,9.999823e-01,9.996749e-01,1.196570e-07,9.998658e-01,9.999715e-01
rxn09201_c0,9.999998e-01,9.999992e-01,0.995081,9.991523e-01,3.074558e-05,9.998569e-01,9.999809e-01,9.989136e-01,9.930420e-01,9.988913e-01
rxn01465_c0,1.000000e+00,9.999971e-01,0.999476,9.967706e-01,9.882901e-01,9.056667e-01,9.999673e-01,9.978272e-01,9.441309e-01,9.112535e-01
rxn29120_c0,3.836529e-08,6.785368e-11,0.000018,1.601663e-09,8.828785e-10,1.144246e-11,9.999978e-01,1.369689e-09,1.146017e-07,1.081941e-11


Of course usually we dont have a dictionary or binary array but rather a model:

In [30]:
model_path = os.path.join(path.parent, 'files',  'models', 'E_coli_KTE31_388739.3_r30_removed.sbml')
draft_model = cobra.io.read_sbml_model(model_path) 


Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-09


We can make a list of the reactions in a model

In [31]:
list_of_reactions = draft_model.reactions.list_attr('id')
NN_MS.predict(input=list_of_reactions)

or just give the model

In [32]:
NN_MS.predict(input=draft_model)

#reactions not found in keys:  73 / 995
[[0 1 0 ... 1 0 0]]


{'rxn08713_c0': 6.017166509622734e-07,
 'rxn03137_c0': 0.9987085900213202,
 'rxn09195_c0': 7.140207812436794e-08,
 'rxn01279_c0': 0.020445076134591813,
 'rxn00720_c0': 0.0006954690713579849,
 'rxn12639_c0': 0.9999999544234285,
 'rxn20214_c0': 4.665460933467111e-07,
 'rxn08076_c0': 0.9998056706770259,
 'rxn01213_c0': 0.00019294259110864972,
 'rxn01537_c0': 0.5166234682923297,
 'rxn03918_c0': 0.020721993270858323,
 'rxn05252_c0': 0.9999756927845834,
 'rxn10317_c0': 3.2372482051132177e-07,
 'rxn23042_c0': 0.9939120360322818,
 'rxn08542_c0': 3.563720308006154e-08,
 'rxn03886_c0': 0.0369121267074663,
 'rxn04751_c0': 0.0004519860014603125,
 'rxn09313_c0': 0.13283582361963958,
 'rxn02296_c0': 9.899144248679522e-07,
 'rxn03241_c0': 0.9999993049095222,
 'rxn05195_c0': 0.9999999905639995,
 'rxn18575_c0': 3.4930027325664185e-07,
 'rxn03540_c0': 0.9831299796119477,
 'rxn05335_c0': 0.9998178547691784,
 'rxn02227_c0': 5.989399266720395e-07,
 'rxn00745_c0': 0.005475607212171035,
 'rxn00510_c0': 0.790